In [3]:
import pandas as pd
import json
import os
from os import listdir
from os.path import isfile, join
import re
import multiprocessing as mp

In [30]:
datapath = os.path.join('C:/', 'data', 'poltweets', 'tweets')

datafiles = [join(datapath,f) for f in listdir(datapath) if isfile(join(datapath, f))]

datafiles = [datafile for datafile in datafiles if re.match(r'.*\.ndjson', datafile)]

filepath = os.path.join('C:/', 'data', 'poltweets', "tweets_flattened_20200115.gz")


In [5]:
def read_datafiles(datafiles):
    df = pd.DataFrame()
    for datafile in datafiles:
        records = map(json.loads, open(datafile, encoding = 'utf-8'))
        df = df.append(pd.DataFrame.from_records(records), ignore_index = True)
    return(df)

def split_job(datafiles):
    with mp.Pool(4) as pool:
        combined_df = pd.DataFrame()
        split = 4
        length = int(len(datafiles)/split) #length of each fold
        folds = []
        for i in range(split-1):
            folds += [datafiles[i*length:(i+1)*length]]
        folds += [datafiles[(split-1)*length:len(datafiles)]]
        
        results = []
        for fold in folds:
            results.append(pool.apply_async(read_datafiles, args = (fold, )))
        
        dfs = [result.get() for result in results]
        for df in dfs:
            combined_df.append(df, ignore_index = True)
            
        return(combined_df)

In [9]:
tweets_df = read_datafiles(datafiles[50:150])

In [206]:
tweets_df.head(10)

,created_at,id,id_str,full_text,truncated,display_text_range,entities,source,in_reply_to_status_id,in_reply_to_status_id_str,...,quoted_status_id_str,quoted_status_permalink,retweet_count,favorite_count,favorited,retweeted,lang,extended_entities,possibly_sensitive,quoted_status
0,Thu Oct 22 13:01:05 +0000 2020,1319262500743975000,1319262500743974914,RT @NHOERUP: Tak for at holde ved @TorstenGejl...,False,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,...,1319217553772859397,"{'url': 'https://t.co/5JGfa4Iicd', 'expanded':...",4,0,False,False,da,NaN,NaN,NaN
1,Thu Oct 22 13:00:50 +0000 2020,1319262436105588700,1319262436105588737,RT @plejefamilierne: Det kan ikke lade sig gør...,False,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,...,NaN,NaN,4,0,False,False,da,NaN,NaN,NaN
2,Thu Oct 22 13:00:32 +0000 2020,1319262363145687000,1319262363145687041,Det skriger til himlen.\nSocialministeren bad ...,False,"[0, 274]","{'hashtags': [{'text': 'dkpol', 'indices': [25...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,...,NaN,NaN,7,53,False,False,da,NaN,NaN,NaN
3,Thu Oct 22 12:32:30 +0000 2020,1319255308875739100,1319255308875739138,RT @alternativet_: Statsministeren sagde i nyt...,False,"[0, 139]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,...,NaN,NaN,4,0,False,False,da,NaN,NaN,NaN
4,Thu Oct 22 11:40:38 +0000 2020,1319242255115391000,1319242255115390982,RT @alternativet_: Vi kan ikke være bekendt at...,False,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,...,NaN,NaN,5,0,False,False,da,NaN,NaN,NaN
5,Thu Oct 22 10:02:29 +0000 2020,1319217553772859400,1319217553772859397,"Har igen, kl. 13 (sammen med V, K og DF) indka...",False,"[0, 279]","{'hashtags': [{'text': 'dkpol', 'indices': [26...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,...,NaN,NaN,8,54,False,False,da,NaN,NaN,NaN
6,Wed Oct 21 21:45:55 +0000 2020,1319032191729488000,1319032191729487879,RT @Formand_DSE: Var i dag på besøg hos @Torst...,False,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,...,NaN,NaN,4,0,False,False,da,NaN,NaN,NaN
7,Wed Oct 21 14:48:39 +0000 2020,1318927184111730700,1318927184111730696,Er på vej i miljøministeriet for at foreslå at...,False,"[0, 276]","{'hashtags': [{'text': 'dkgreen', 'indices': [...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,...,NaN,NaN,13,47,False,False,da,"{'media': [{'id': 1318927178675998700, 'id_str...",False,NaN
8,Wed Oct 21 13:21:20 +0000 2020,1318905207103914000,1318905207103913985,Fremragende foretræde af tidligere STU elev Da...,False,"[0, 275]","{'hashtags': [{'text': 'enmillionstemmer', 'in...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,...,NaN,NaN,4,29,False,False,da,"{'media': [{'id': 1318905153483886600, 'id_str...",False,NaN
9,Tue Oct 20 17:52:55 +0000 2020,1318611166843850800,1318611166843850752,@PerVismark @politikerpip @DenGroenneStud @con...,False,"[66, 142]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/iphone"" r...",1.31857e+18,1318565907954884610,...,NaN,NaN,0,10,False,False,da,NaN,NaN,NaN


In [205]:
pd.json_normalize(tweets_df.loc[0:5, 'in_reply_to_status_id'])

AttributeError: 'float' object has no attribute 'values'

In [14]:
chunked_data = chunk_data(datapath)

In [53]:
select_columns = ['created_at', 'id', 'full_text', 'truncated', 'display_text_range', 'source', 'in_reply_to_status_id', 'in_reply_to_user_id', 
                  'in_reply_to_screen_name', 'user', 'contributors', 'is_quote_status', 'quoted_status_id', 'quoted_status_permalink', 'retweet_count', 
                  'favorite_count', 'retweeted', 'quoted_status', 'name', 'hashtags', 'urls']

In [54]:
tweets_sel_df = tweets_df.loc[:, select_columns]

In [55]:
tweets_sel_df = tweets_sel_df.sample(frac=0.05, replace=False, random_state=42)

In [58]:
tweets_sel_df.to_csv('../../data/tweets/tweets_combined_sample_20201201.csv')